## Tokenization

In [ ]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
    verdict = f.read()

import re
alltext = re.findall(r'\w+|[,.!?"\']|--',verdict)


alltext = sorted(set(alltext)) #sort the text

print(alltext)

vocab = {key:value for key,value in enumerate(alltext)} ## vocab
# for i,item in enumerate(vocab.items()):
#     if(i<50):
#         print(i,item)



In [ ]:
class SimpleClassTokenizer:
    def __init__(self,vocab):
        self.str_to_int = {s:i for i,s in vocab.items()}
        self.int_to_str = vocab
    
    def encode(self,text):
        preprocessed = re.findall(r'\w+|[,.!?"\']|--',text)
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
     
    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.!?"()\'])', r'\1', text)
        return text


In [ ]:
tokenizer = SimpleClassTokenizer(vocab)

text1 = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text1)
print(ids)
tokenizer.decode(ids)


In [ ]:
# print(alltext)
alltext2 = alltext
alltext2.extend(['<|endoftext|>','<|unk|>']) ## Add endline
print(len(alltext2))
vocab = {i:s for i,s in enumerate(alltext2)}
print(len(vocab))


In [ ]:
alltext.clear()

In [ ]:
class SimpleClassTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int = {s:i for i,s in vocab.items()}
        self.int_to_str = vocab

    def encoding(self,text):
        preprocessed = re.findall(r'\w+|--|[,.:;\-()?!"\']',text)
        preprocessed = [item if item in self.str_to_int else '<|endoftext|>' for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decoding(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+(--|[,.:;\-()?!"\'])',r'\1',text)
        return text


In [ ]:
tokenizer = SimpleClassTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text0 = " <|endoftext|> ".join((text1, text2))

print(text0)
ids = tokenizer.encoding(text0)
print(ids)
print(tokenizer.decoding(ids))

## Byte Pair Encoding

In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')
token_ids = tokenizer.encode(verdict,allowed_special={"<|endoftext|>"})
text = tokenizer.decode(token_ids)
print(len(token_ids))
print(len(text))
print(text[:50])

## Create Input and Output Pairs

In [5]:
with open('the-verdict.txt',encoding='utf-8') as f:
    rawdata = f.read()
print(len(rawdata))

import tiktoken
import torch
from torch.utils.data import Dataset,DataLoader

class SimpleClassDataset(Dataset):
    def __init__(self,txt,tokenizer,stride, maxlength):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt,allowed_special={'<|endoftext|>'})

        for i in range(0,len(token_ids)-maxlength,stride):
            input_chunk = token_ids[i:i+maxlength]
            target_chunk = token_ids[i+1:i+maxlength+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index],self.target_ids[index]
    
def create_dataloader(text,stride=128,batch_size=4,max_length=256,shuffle=True,drop_last = True,num_worker=0):

    tokenizer = tiktoken.get_encoding('gpt2')
    
    dataset = SimpleClassDataset(text,tokenizer,stride,max_length)

    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_worker
    )
    
    return dataloader


dataloader = create_dataloader(text=rawdata,batch_size=6,stride=4,max_length=4,shuffle=False,drop_last=True)

data_iter = iter(dataloader)
input,target = next(data_iter)
print(input)
print(target)
print(len(dataloader))



20479
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645]])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049]])
214


## Token Embedding

In [ ]:
torch.manual_seed(123)
embedding_layer= torch.nn.Embedding(50257,256)
print(input.shape)
token_embedding = embedding_layer(input)
print(token_embedding.shape)

torch.Size([4, 4])
torch.Size([4, 4, 256])


## Positional Embedding

In [ ]:
max_length = 4
dimensions = 256
pos_embedding_layer = torch.nn.Embedding(max_length,dimensions)
pos_embedding = pos_embedding_layer(torch.arange(max_length))
print(pos_embedding)

tensor([[ 0.5423, -0.1224, -1.4150,  ...,  0.2515, -2.3067,  0.8155],
        [-0.3973, -1.2575, -1.9800,  ..., -0.1207,  0.3075, -0.6422],
        [ 0.1840,  1.1128,  1.0052,  ...,  0.2081,  0.5531, -1.1619],
        [ 1.4155,  0.6599,  0.3760,  ...,  0.7034, -0.6108,  0.1080]],
       grad_fn=<EmbeddingBackward0>)


## Input Embedding

In [15]:
input_embeddings = token_embedding + pos_embedding
print(input_embeddings.shape)

torch.Size([4, 4, 256])


## Simple Self-Attention

In [1]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [3]:
attention_score = inputs @ inputs.T
attention_weights = torch.softmax(attention_score, dim = -1)
print(attention_weights)
print(attention_weights.sum(dim=-1))

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [7]:
all_context_vector = attention_weights @ inputs
print(all_context_vector)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


## Self-Attention

In [ ]:
import torch 
import torch.nn as nn
class SelfAttention(nn.Module):
    def __init__(self, d_in,d_out,qkv_bias):
        super().__init__()
        self.w_query = nn.Linear(d_in,d_out,qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,qkv_bias)
        self.W_value = nn.Linear(d_in,d_out,qkv_bias)

    
    def forward(self,x):
        query = self.w_query(x)
        key = self.W_key(x)
        value = self.W_value(x)
        attention_score = query @ key.T
        attention_weights = torch.softmax(attention_score/key.shape[-1]**0.5,dim=-1)
        context_vector = attention_weights @ value

        return context_vector
    

## Casual Attention

Text<br>
↓<br>
Tokenization --- (length)<br>
↓<br>
DataLoader -----(batch_size, contextlength) (4,6)<br>
↓<br>
Token Embedding ---(batch_size,context_length,dimensions) (4,6,512) + <br>
Positional Embedding --(batch_size,context_length,dimensions) (4,6,512)<br>
↓<br>
Query,Key,Value weights---(dimensions,attention_head)/(d_in,d_out) +<br>
Query,Key,Value ---(batch,context_length,d_out)<br>
Attention_score ---(batch,context_length,context_length)<br>
context_vector ---(batch,token,d_out)<br>
Self-Attention---(batch,tokens,d_out) <br>



In [ ]:
import torch
import torch.nn as nn
class CasualAttention(nn.Module):
    def __init__(self, d_in,d_out,context_length,qkv_bias,dropout):
        super().__init__()
        self.d_out = d_out
        self.w_query = nn.Linear(d_in,d_out,qkv_bias) ## d_in should be equal to the dimensions of the embedding
        self.w_key = nn.Linear(d_in,d_out,qkv_bias)
        self.w_value = nn.Linear(d_in,d_out,qkv_bias)
        self.drop = nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    def forward(self,x):
        b, num_tokens, d_in = x.shape
        query = self.w_query(x) 
        key = self.w_key(x)
        value = self.w_value(x)

        attention_score = query @ key.transpose(-1,-2) 
        attention_score.masked_fill_(self.mask.bool()[:num_tokens,:num_tokens],-torch.inf) ## PyTorch broadcasting rules
        attention_weights = torch.softmax(attention_score/key.shape[-1]**0.5,dim=-1)
        attention_weights = self.drop(attention_weights) # New

        context_vec = attention_weights @ value
        return context_vec

    
        

## Multi Head Attention Wrapper Class

In [ ]:
class multiheadwrapper(nn.Module):
    ## head_dim = d_out/num_heads
    def __init__(self,d_in,d_out,context_length,num_heads,dropout,qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CasualAttention(d_in,d_out,context_length,qkv_bias,dropout)
             for _ in range(num_heads)]   
        )

    def forward(self,x):
        return torch.cat([head(x) for head in self.heads],dim =-1) ## concat by increasing the dimensions of the token

## Multihead Attention 

In [ ]:
import torch
import torch.nn as nn
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in,d_out,context_length,num_heads,dropout,qkv_bias= False):
        super().__init__()
        assert(d_out%num_heads==0),"d_out should be divisible by the number of heads"
        self.d_out = d_out  ## d_out = 768
        self.num_heads = num_heads ## num_head = 12
        self.head_dim = d_out // num_heads ## head_dim = 64
        self.w_query = nn.Linear(d_in,d_out,qkv_bias)
        self.w_key = nn.Linear(d_in,d_out,qkv_bias)
        self.w_value = nn.Linear(d_in,d_out,qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    def forward(self,x):

        b,num_token,d_in = x.shape

        query = self.w_query(x)  ## (batch,token,d_out)
        key = self.w_key(x)
        value = self.w_value(x)

        query = query.view(b,num_token,self.num_heads,self.head_dim)
        key = key.view(b,num_token,self.num_heads,self.head_dim)
        value = value.view(b,num_token,self.num_heads,self.head_dim)

        query = query.transpose(1,2) ## (batch,num_heads,token,head_dim)
        key = key.transpose(1,2)
        value = value.transpose(1,2)

        attention_score = query @ key.transpose(2,3) ## (batch,num_heads,token,token)
        attention_score.masked_fill_(self.mask.bool()[:num_token,:num_token],-torch.inf)
        attention_weights = torch.softmax(attention_score/key.shape[-1]**0.5,dim=-1) 
        attention_weights = self.dropout(attention_weights)
        context = (attention_weights @ value)
        context = context.transpose(1, 2).contiguous()
        context = context.view(b, num_token, self.d_out)
        
        return self.out_proj(context)   ##(batch,token,d_out)


In [9]:
torch.manual_seed(123)

# Define the tensor with 3 rows and 6 columns
inputs = torch.tensor(
    [[0.43, 0.15, 0.89, 0.55, 0.87, 0.66],  # Row 1
     [0.57, 0.85, 0.64, 0.22, 0.58, 0.33],  # Row 2
     [0.77, 0.25, 0.10, 0.05, 0.80, 0.55]]  # Row 3
)

batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

batch_size, context_length, d_in = batch.shape
d_out = 6
mha = MultiHeadAttention(d_in, d_out, context_length, 2, 0.0)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 3, 6])
tensor([[[ 0.1914, -0.1715,  0.0275, -0.0691, -0.2708, -0.2775],
         [ 0.0885, -0.0364, -0.0123, -0.0215, -0.3108, -0.2577],
         [ 0.2031, -0.0604,  0.0302,  0.0702, -0.3894, -0.2861]],

        [[ 0.1914, -0.1715,  0.0275, -0.0691, -0.2708, -0.2775],
         [ 0.0885, -0.0364, -0.0123, -0.0215, -0.3108, -0.2577],
         [ 0.2031, -0.0604,  0.0302,  0.0702, -0.3894, -0.2861]]],
       grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 3, 6])


## Layer Normalisation

In [3]:
import torch
import torch.nn as nn
class LayerNormalisation(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    def forward(self,x):
        mean = x.mean(dim=-1,keepdim=True)
        var = x.var(dim = -1,keepdim=True,unbiase=False)
        nor = (x-mean)/torch.sqrt(var+self.eps)
        return self.scale * nor + self.shift


## Gelu Activation

In [ ]:
import torch 
import torch.nn as nn

class GeluActivation(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return (0.5 * x)*(1+torch.tanh(torch.sqrt(torch.tensor(2.0/torch.pi))*(x+0.044715*torch.pow(x,3))))

## Feed Forward Network

In [ ]:
import torch 
import torch.nn as nn

class FeedForward(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), ## Expansion
            GeluActivation(), ## Activation
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]), ## Contraction
        )
    def forward(self,x):
        return self.layers(x)

In [5]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

## Transformer Block

In [ ]:
import torch
import torch.nn as nn

class Transformer(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.layernor = LayerNormalisation(cfg['emb_dim'])
        self.drop = nn.Dropout(cfg['drop_rate'])
        self.feedfor = FeedForward(cfg)
    def forward(self,x):
        shortcut = x
        x = self.layernor(x)
        x = self.att(x)
        x = self.drop(x)
        x = x + shortcut

        shortcut = x
        x = self.layernor(x)
        x = self.feedfor(x)
        x = self.drop(x)
        x = x + shortcut

        return x


## GPT Model

In [ ]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[Transformer(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNormalisation(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();  # Disable dropout during inference

## Cross-Entrophy loss

## GPT Architecture

In [1]:
def cross_entrophy_loss(input_batch,target_batch,device,model):
    input_batch,target_batch = input_batch.to(device),target_batch.to(device)
    logits = model(input_batch)
    loss = nn.functional.cross_entropy(logits.flatten(0,1),target_batch.flatten())
    return loss

def cross_loss_dataloader(dataloader,num_batches,device,model):
    if len(dataloader) == 0:
        return float('nan')
    elif(num_batches is None):
        num_batches = len(dataloader)
    else:
        total_loss = 0
        num_batches = torch.min(num_batches,len(dataloader))

    for i,(input_batches,target_batches) in enumerate(dataloader):
        if(i<num_batches):
                loss = cross_entrophy_loss(input_batch=input_batches,target_batch=target_batches,device=device,model = model)
                total_loss +=loss.item
        else:
            break
    return total_loss / num_batches






In [2]:
with open ('the-verdict.txt',encoding='utf-8') as f:
    raw_data = f.read()

import torch
import torch.nn as nn
import tiktoken
from torch.utils.data import Dataset,DataLoader

class Datasetv1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):

        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt,allowed_special={'|<endoftext>|'})

        for i in range (0 , len(token_ids)-max_length,stride): ## create a window that goes past the end of the token list.
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
        
    def __getitem__(self,x):
        return self.input_ids[x], self.target_ids[x]
    

def Create_Dataloader(text,batch_size,stride,max_length,shuffle,drop_last,num_workers):
     
    tokenizer = tiktoken.get_encoding('gpt2')

    dataset = Datasetv1(text,tokenizer,max_length,stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers= num_workers
    )

    return dataloader


        

In [3]:
train_ratio = 0.90
split_indx = int(train_ratio * len(raw_data))
train_data = raw_data[:split_indx]

valid_data = raw_data[:split_indx - len(raw_data)]

GPT_CONFIG_124M={
    'vocab_size':50257,
    'context_length':1024,
    'n_layers':12,
    'n_heads':12,
    'emb_dimension':768,
    'drop_rate':0.1,
    'qkv_bias':False
}

train_loader = Create_Dataloader(
    text=train_data,
    batch_size=2,
    stride=GPT_CONFIG_124M['context_length'],
    max_length=GPT_CONFIG_124M['context_length'],
    drop_last=True,
    shuffle=True,
    num_workers=0

)

test_loader = Create_Dataloader(
    text=valid_data,
    batch_size=2,
    stride=GPT_CONFIG_124M['context_length'],
    max_length=GPT_CONFIG_124M['context_length'],
    drop_last=False,
    shuffle=False,
    num_workers=0

)

data_iter = iter(train_loader)
input,target = next(data_iter)
print(input)
input2,target2 = next(data_iter)
print(input2)
print(len(train_loader))
print(input.shape)
for input,target in train_loader:
    print(input)

tensor([[   40,   367,  2885,  ...,   691, 12226,   318],
        [  284,  1234,  8737,  ...,   402,   271, 10899]])
tensor([[ 1310,  1165,   881,  ...,   284,   616,   835],
        [   11, 17728,   257,  ...,   423,  1309,   257]])
2
torch.Size([2, 1024])
tensor([[   40,   367,  2885,  ...,   691, 12226,   318],
        [  284,  1234,  8737,  ...,   402,   271, 10899]])
tensor([[   11, 17728,   257,  ...,   423,  1309,   257],
        [ 1310,  1165,   881,  ...,   284,   616,   835]])


In [4]:
class LayerNormalisation(nn.Module):
    def __init__(self,GPT_CONFIG_124M):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(GPT_CONFIG_124M['emb_dimension']))
        self.shift = nn.Parameter(torch.zeros(GPT_CONFIG_124M['emb_dimension']))

    def forward(self,x):
        mean_x = x.mean(dim=-1, keepdim = True)
        var_x = x.var(dim = -1,keepdim = True,unbiased = False)
        nor_x = (x - mean_x)/torch.sqrt(var_x+self.eps)

        return nor_x * self.scale + self.shift
    
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
        return (0.5*x)*(1+torch.tanh(
            torch.sqrt(torch.tensor(2/torch.pi))
            *(x+0.044715 *torch.pow(x,3))))
    
class FeedForward(nn.Module):
    def __init__(self,GPT_CONFIG_124M):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(GPT_CONFIG_124M['emb_dimension'],GPT_CONFIG_124M['emb_dimension']*4),
            GELU(),
            nn.Linear(GPT_CONFIG_124M['emb_dimension']*4,GPT_CONFIG_124M['emb_dimension'])
        )
    def forward(self,x):
        return self.layers(x)

class MultiheadAttention(nn.Module):
    def __init__(self,GPT_CONFIG_124M):
        super().__init__()
        assert(GPT_CONFIG_124M['emb_dimension'] % GPT_CONFIG_124M['n_heads'] == 0), "Not compatible Parameters"
        self.n_heads = GPT_CONFIG_124M['n_heads']
        self.d_out = GPT_CONFIG_124M['emb_dimension']
        self.h_dim = self.d_out // self.n_heads
        self.w_query = nn.Linear(GPT_CONFIG_124M['emb_dimension'],GPT_CONFIG_124M['emb_dimension'],bias=False)
        self.w_key = nn.Linear(GPT_CONFIG_124M['emb_dimension'],GPT_CONFIG_124M['emb_dimension'],bias=False)
        self.w_value = nn.Linear(GPT_CONFIG_124M['emb_dimension'],GPT_CONFIG_124M['emb_dimension'],bias=False)
        self.register_buffer('mask',torch.triu(torch.ones(GPT_CONFIG_124M['context_length'],GPT_CONFIG_124M['context_length']),diagonal=1))
        self.drop = nn.Dropout(GPT_CONFIG_124M['drop_rate'])
        self.out_proj = nn.Linear(GPT_CONFIG_124M['emb_dimension'],GPT_CONFIG_124M['emb_dimension'])
    def forward(self,x):
        batch,token,emb_dim = x.shape
        query = self.w_query(x)
        key = self.w_key(x)
        value = self.w_value(x)

        query = query.view(batch,token,self.n_heads,self.h_dim)
        key = key.view(batch,token,self.n_heads,self.h_dim)
        value = value.view(batch,token,self.n_heads,self.h_dim)

        query = query.transpose(1,2)
        key = key.transpose(1,2)
        value = value.transpose(1,2)

        attention_score = query @ key.transpose(2,3)
        attention_score.masked_fill_(self.mask.bool()[:token,:token],-torch.inf)
        attention_weights = torch.softmax(attention_score/key.shape[-1]**0.5,dim=-1)
        attention_weights = self.drop(attention_weights)
        context_vecs = attention_weights @ value
        context_vecs = context_vecs.transpose(1,2)
        context_vecs = context_vecs.contiguous().view(batch,token,self.d_out)

        return self.out_proj(context_vecs)
        
class Transformer(nn.Module):
    def __init__(self,GPT_CONFIG_124M):
        super().__init__()
        self.layer_nor = LayerNormalisation(GPT_CONFIG_124M)
        self.multi_head = MultiheadAttention(GPT_CONFIG_124M)
        self.feed_for = FeedForward(GPT_CONFIG_124M)
        self.drop = nn.Dropout(GPT_CONFIG_124M['drop_rate'])
    def forward(self,x):
        shortcut = x
        x = self.layer_nor(x)
        x = self.multi_head(x)
        x = self.drop(x)
        x = x+shortcut

        shortcut = x
        x = self.layer_nor(x)
        x = self.feed_for(x)
        x = self.drop(x)
        x=x+shortcut

        return x
    
class GPT_ARCHITECTURE(nn.Module):
    def __init__(self,GPT_CONFIG_124M):
        super().__init__()
        self.embedding_layer = nn.Embedding(GPT_CONFIG_124M['vocab_size'],GPT_CONFIG_124M['emb_dimension'])
        self.positional_layer = nn.Embedding(GPT_CONFIG_124M['context_length'],GPT_CONFIG_124M['emb_dimension'])
        self.drop = nn.Dropout(GPT_CONFIG_124M['drop_rate'])
        self.transformer = nn.Sequential(*[Transformer(GPT_CONFIG_124M) for _ in range(GPT_CONFIG_124M['n_layers'])])
        self.linear = nn.Linear(GPT_CONFIG_124M['emb_dimension'],GPT_CONFIG_124M['vocab_size'])
        self.layer_nor = LayerNormalisation(GPT_CONFIG_124M)

    def forward(self,x):
        batch_size,seq_length = x.shape

        token_embedding = self.embedding_layer(x)
        positional_embedding = self.positional_layer(torch.arange(seq_length,device = x.device).unsqueeze(0))

        input_embedding = token_embedding + positional_embedding
        x = self.drop(input_embedding)
        x= self.transformer(x)
        x=self.layer_nor(x)
        logits=self.linear(x)
    
        return logits


In [5]:
train_token = 0
valid_token = 0
for input_batch,target_batch in train_loader:
    print(input_batch.shape)
    train_token +=input_batch.numel()


for input_batch,target_batch in test_loader:
    valid_token += input_batch.numel()

torch.manual_seed(123)
model = GPT_ARCHITECTURE(GPT_CONFIG_124M)

def cross_entrophy_loss(input_batch,target_batch,device,model):
    input_batch,target_batch = input_batch.to(device),target_batch.to(device)
    logits = model(input_batch)
    loss = nn.functional.cross_entropy(logits.flatten(0,1),target_batch.flatten())
    return loss

def cross_loss_dataloader(dataloader,device,model,num_batches = None):
    total_loss = 0

    if len(dataloader) == 0:
        return float('nan')
    elif(num_batches is None):
        num_batches = len(dataloader)
    else:
        num_batches = torch.min(num_batches,len(dataloader))

    for i,(input_batches,target_batches) in enumerate(dataloader):
        if(i<num_batches):
                loss = cross_entrophy_loss(input_batch=input_batches,target_batch=target_batches,device=device,model = model)
                total_loss +=loss.item()
        else:
            break
    return total_loss / num_batches






torch.Size([2, 1024])
torch.Size([2, 1024])


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes


torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = cross_loss_dataloader(dataloader=train_loader, model=model, device=device)
    val_loss = cross_loss_dataloader(dataloader=train_loader,model= model, device=device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)


Training loss: 11.000597953796387
Validation loss: 10.999808311462402


In [9]:
def text_to_token(txt,tokenizer):
    token_ids = tokenizer.encode(txt,allowed_special={'<|endoftext|>'})
    ids = torch.tensor(token_ids).unsqueeze(0)
    return ids

def token_to_text(ids,tokenizer):
    total_ids = ids.squeeze(0)
    text = tokenizer.decode(total_ids.tolist())
    return text

def generate_text_simple(model,token_ids,max_token,context_length):

    for _ in range(max_token):
        idx = token_ids[:,-context_length:]

        with torch.no_grad():
            logits = model(idx)

        next_logits = logits[:,-1,:]

        probas = torch.softmax(next_logits,dim=-1)
        next_ids = torch.argmax(probas,dim = -1,keepdim=True)
        idx = torch.cat((idx,next_ids),dim=-1)

    return idx


text = "Hi, how are you ?"
tokenizer = tiktoken.get_encoding('gpt2')
token_ids = generate_text_simple(model=model,
                                 token_ids=text_to_token(text,tokenizer),
                                 max_token=4,
                                 context_length=5)

full_sentence = token_to_text(token_ids,tokenizer)


